# REDA Example Notebook

In [ ]:
# !pip install git+git://github.com/geophysics-ubonn/reda

In [ ]:
%matplotlib inline
import reda.utils.mpl
plt, mpl = reda.utils.mpl.setup()

import numpy as np
np.random.seed(2017)

from reda import ERT
obj = ERT()
import glob
for nr, filename in enumerate(sorted(
        glob.glob('data/pygimli_*.ohm'))):
    obj.import_bert(filename, timestep=nr)
    
obj.compute_K_analytical(spacing=1)

In [ ]:
print(obj.data.head(10))

## Filtering data

In [ ]:
# apply to all timesteps
# obj.filter('K > 1000')

In [ ]:
# apply only to timestep 10
obj.sub_filter(
    'timestep == 10',
    'K > 1000',    
)

In [ ]:
print(obj.data.query('timestep == 10 and rho_a < 200'))

## Histogram plotting

In [ ]:
import reda.plotters.histograms as RH
fig = RH.plot_histograms_extra_dims(
    obj,
    keys=['rho_a'],
    extra_dims=['timestep', ],
    Nx=5,
    #subquery='timestep in {0}'.format(list(range(0, 100, 10))),
    subquery='timestep in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]'
)
print(fig.get_figwidth(), fig.get_figheight())
fig.savefig('histogram.png', dpi=600)

## Plotting Pseudosections

In [ ]:
import reda.plotters.pseudoplots as PS
fig = PS.plot_ps_extra(
    obj, 'rho_a',
    subquery='timestep in [0, 10, 20, 30, 40, 50, 60, 70, 80, 90]'
)
fig.set_size_inches(7.87, 3.93)
fig.tight_layout()
fig.savefig('pseudoplots.png', dpi=600)

## Analyzing time series

In [ ]:
import reda.plotters.time_series as TS

fig, ax = TS.plot_quadpole_evolution(
    obj,
    [10, 11, 15, 14],
    'rho_a',
    threshold=0.05,
    rolling=True,
)

In [ ]:
# add some noise to the data
import numpy as np
np.random.seed(2017)
obj.data['rho_a'] = obj.data['rho_a'] + (np.abs(obj.data['rho_a'] * 0.05 * np.random.randn(obj.data.shape[0])))

In [ ]:
fig, ax = TS.plot_quadpole_evolution(
    obj,
    [10, 11, 15, 14],
    'rho_a',
    threshold=0.05,
    rolling=True,
)

## The Data Journal

In [ ]:
obj.print_data_journal()